### SmartMeter 2012 Simulations

Simulations with varying number of LSTM neurons (x2, x4, and x10) and cross enthropy as the loss function.


In [7]:
# Import the dependencies
import pandas as pd
import numpy as np
from Input_prep import train_val_test
import os
from tensorflow.keras import losses
from keras import layers, Sequential, optimizers
from utils import create_sequences






In [3]:
# Create lists for neuron multipliers and loss functions
neurons_list = [2, 4, 10]

loss_functions = {
    'MSE': losses.MeanSquaredError(),
    'MAE': losses.MeanAbsoluteError(),
    'Huber (δ=1.5)': losses.Huber(delta=1.5),
    'MSLE': losses.MeanSquaredLogarithmicError(),
    'Cross Entropy': losses.BinaryCrossentropy()
}

data_points_drop = [0.1, 0.2, 0.3]

VERBOSE = 1

num_meters_list = [50, 100, 4411]

models = ['LSTM_Simple', 'LSTM_stacked']




In [ ]:
# import the data
dataset = pd.read_csv("D:/FL Publication/Code_new/FL_Publication_1/Current_Simulations/Data_Preprocessing/Preprocessed_data/SmartMeter_2013_hourly.csv")

data_splits = {
    'train': (dataset["train"].drop(columns=['KWH/hh (per hour)']),
            dataset["train"]['KWH/hh (per hour)']),
    'validation': (dataset["validation"].drop(columns=['KWH/hh (per hour)']),
                dataset["validation"]['KWH/hh (per hour)']),
    'test': (dataset["test"].drop(columns=['KWH/hh (per hour)']),
            dataset["test"]['KWH/hh (per hour)'])
}


In [ ]:
# Define the models

def get_LSTM_Simple(time_steps, num_features, learning_rate, neurons=60, loss_function='mean_squared_error'):
    model = Sequential()
    model.add(layers.Input(shape=(time_steps, num_features)))
    model.add(layers.LSTM(neurons, return_sequences=True))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1))

    optimizer=optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['mean_absolute_error'])
    return model


def get_LSTM_stacked(time_steps, num_features, learning_rate, neurons=60, loss_function='mean_squared_error'):
    model = Sequential()
    model.add(layers.Input(shape=(time_steps, num_features)))
    model.add(layers.LSTM(neurons//2, return_sequences=True)) 
    model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(neurons//3, return_sequences=True))
    model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(neurons//4, return_sequences=True))
    model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(neurons//12))  
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1))

    optimizer = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['mean_absolute_error'])
    return model


In [ ]:
# Run the each model in a loop  with the different loss functions and neuron multipliers
for num_meters in num_meters_list:
    for neuron_multiplier in neurons_list:
        for loss_function in loss_functions:
            for data_dropout in data_points_drop: # Added data dropout loop
                for model_name in models:
                    if model_name == 'LSTM_Simple':
                        data_splits = {
                            'train': (df_dataset["train"].drop(columns=[target_column]),
                                    df_dataset["train"][target_column]),
                            'validation': (df_dataset["validation"].drop(columns=[target_column]),
                                        df_dataset["validation"][target_column]),
                            'test': (df_dataset["test"].drop(columns=[target_column]),
                                    df_dataset["test"][target_column])
                        }
                    
                        # Convert all data to numpy arrays
                        for split in data_splits:          
                            data_splits[split] = (np.asarray(data_splits[split][0]),
                                                np.asarray(data_splits[split][1]))
                            
                        # Extract features and labels for training, validation, and testing
                        x_train, y_train = data_splits['train']
                        x_val, y_val = data_splits['validation']
                        x_test, y_test = data_splits['test']
                        
                        # Apply data dropout if specified
                        if data_dropout > 0:
                            mask = np.random.rand(*x_train.shape) > data_dropout
                            x_train = x_train * mask
                        
                        if model_name in ['simple_LSTM', 'stacked_LSTM', 'bidirectional_LSTM', 'GRU']:
                            try:
                                # Create sequences
                                x_train_seq, y_train_seq = create_sequences(
                                    X = x_train, y = y_train, time_steps = time_steps)
                                x_val_seq, y_val_seq = create_sequences(
                                    X = x_val, y = y_val, time_steps = time_steps)
                                x_test_seq, y_test_seq = create_sequences(
                                    X = x_test, y = y_test, time_steps = time_steps)

                                num_features = x_train_seq.shape[2]

                                # Initialize model with time_steps and num_features
                                model = model_func(time_steps=time_steps, num_features=num_features, **model_params)

                                # Convert data to tensors
                                x_train_tensor = convert_to_tensor(x_train_seq, dtype=tf.float32)
                                y_train_tensor = convert_to_tensor(y_train_seq, dtype=tf.float32)
                                x_val_tensor = convert_to_tensor(x_val_seq, dtype=tf.float32)
                                y_val_tensor = convert_to_tensor(y_val_seq, dtype=tf.float32)
                                x_test_tensor = convert_to_tensor(x_test_seq, dtype=tf.float32)
                                y_test_tensor = convert_to_tensor(y_test_seq, dtype=tf.float32)

                                # Add early stopping callback
                                early_stopping = tf.keras.callbacks.EarlyStopping(
                                    monitor='val_loss',
                                    patience=3,
                                    restore_best_weights=True,
                                )
                                
                                # Train the model
                                history = model.fit(
                                    x_train_tensor,
                                    y_train_tensor,
                                    validation_data=(x_val_tensor, y_val_tensor),
                                    epochs=10,
                                    batch_size=512,
                                    verbose=verbose,
                                    callbacks=[early_stopping]
                                )
                                
                                training_time = time.time() - start_time
                                results = model.evaluate(x_test_tensor, y_test_tensor, verbose=verbose)
                            
                                return results, history.history, training_time, num_meters

                        model = get_LSTM_Simple(time_steps, num_features, learning_rate, 60 * neuron_multiplier, loss_function)
                    elif model_name == 'LSTM_stacked':
                        model = get_LSTM_stacked(time_steps, num_features, learning_rate, 60 * neuron_multiplier, loss_function)



        

